In [110]:
import pandas as pd
import csv
import math
from datetime import datetime
from dateutil.relativedelta import relativedelta
import numpy as np

In [111]:
stocks_df = pd.read_csv('../data_harvesting/amundi-msci-wrld-ae-c.csv')
cbonds_df = pd.read_csv('../data_harvesting/ishares-global-corporate-bond.csv')
sbonds_df = pd.read_csv('../data_harvesting/db-x-trackers-ii-global-sovereign-5.csv')
gold_df = pd.read_csv('../data_harvesting/spdr-gold-trust.csv')
usd_df = pd.read_csv('../data_harvesting/usdollar.csv')

portfolio_df = pd.read_csv('../portfolio_allocations/portfolio_allocations.csv')

In [112]:
def previous_neighbor(L, n):
    m = 0
    j = 0
    for i, x in enumerate(L):
        if x < n and x > m:
            m = x
            j = i
    return j

def missing_value(df, month):
    df_month = df[df['Date'].str.contains(month)]

    days = []
    for index, row in df_month.iterrows():
        days.append(int(row[0][4:6]))
    
    if 15 not in days:
        index_change = previous_neighbor(days, 15) + df_month.index[0]
        new_value = df_month.iloc[len(df_month)//2]['Date'][0:3] + ' 15' + df_month.iloc[len(df_month)//2]['Date'][6:]
        df.at[index_change, 'Date'] = new_value


def replace():
    files = [stocks_df, cbonds_df, sbonds_df, gold_df, usd_df]
    months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    
    for file in files:
        for month in months:
            missing_value(file, month)
    



In [113]:
replace()

In [130]:
#input data
amount = 100000
inv_date = datetime(2020, 7, 1)
inv_period = 3

print(sbonds_df)
print(portfolio_df.head())

#prices
stocks_index = stocks_df.loc[stocks_df['Date'] == inv_date.strftime('%b %d, %Y')]
if stocks_index.empty:
        print("No price data for date, stocks!")
stocks_price = stocks_index.iloc[0]['Price']

cbonds_index = cbonds_df.loc[cbonds_df['Date'] == inv_date.strftime('%b %d, %Y')]
if cbonds_index.empty:
        print("No price data for date, cbonds!")
cbonds_price = cbonds_index.iloc[0]['Price']

sbonds_index = sbonds_df.loc[sbonds_df['Date'] == inv_date.strftime('%b %d, %Y')]
if sbonds_index.empty:
        print("No price data for date!, sbonds")
sbonds_price = sbonds_index.iloc[0]['Price']

gold_index = gold_df.loc[gold_df['Date'] == inv_date.strftime('%b %d, %Y')]
if gold_index.empty:
        print("No price data for date, gold!")
gold_price = gold_index.iloc[0]['Price']

usd_price = 1

print("stocks price:", stocks_price)
print("cbonds price:", cbonds_price)
print("sbonds price:", sbonds_price)
print("gold price:", gold_price)



             Date   Price    Open    High     Low   Vol. Change %
0    Dec 30, 2020  253.84  253.82  253.95  253.42  1.98K    0.01%
1    Dec 29, 2020  253.82  253.70  253.82  253.41  1.69K    0.11%
2    Dec 28, 2020  253.55  254.18  254.29  253.50  1.49K   -0.25%
3    Dec 23, 2020  254.18  254.90  255.10  253.67  3.36K   -0.37%
4    Dec 22, 2020  255.13  254.47  255.13  253.33  4.34K    0.28%
..            ...     ...     ...     ...     ...    ...      ...
250  Jan 08, 2020  254.82  254.80  254.82  254.28  1.13K    0.18%
251  Jan 07, 2020  254.36  253.66  254.60  253.63  0.91K    0.08%
252  Jan 06, 2020  254.16  254.42  254.42  254.40  0.05K   -0.38%
253  Jan 03, 2020  255.12  254.73  255.62  254.41  3.72K    0.73%
254  Jan 02, 2020  253.28  252.28  253.55  252.02  0.47K    0.29%

[255 rows x 7 columns]
   Unnamed: 0    ST    CB    PB    GO    CA
0           0  1.00  0.00  0.00  0.00  0.00
1           1  0.95  0.05  0.00  0.00  0.00
2           2  0.95  0.00  0.05  0.00  0.00
3       

In [131]:
methodsFile = '../trading_methodologies/trading_methodologies.csv'
with open(methodsFile, 'w', newline = '') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(["Date", "Trading method", "Allocation no", "Asset", "Weight of asset", "Asset price", "Amount to buy", "Total spending", "Investment period"])

#one-off, no rebalance

for index, row in portfolio_df.iterrows():

    allocation_index = row[0]
    stocks_rate = row['ST']*amount
    cbonds_rate = row['CB']*amount
    sbonds_rate = row['PB']*amount
    gold_rate = row['GO']*amount
    usd_rate = row['CA']*amount


        #number of assets to buy from each
    stocks_amt = math.floor(stocks_rate/stocks_price)
    cbonds_amt = math.floor(cbonds_rate/cbonds_price)
    sbonds_amt = math.floor(sbonds_rate/sbonds_price)
    gold_amt = math.floor(gold_rate/gold_price)
    usd_amt = math.floor(usd_rate/usd_price)


        #amount of money to invest in each

    stocks_spend = round(stocks_amt*stocks_price,2)
    cbonds_spend = round(cbonds_amt*cbonds_price,2)
    sbonds_spend = round(sbonds_amt*sbonds_price,2)
    gold_spend = round(gold_amt*gold_price,2)
    usd_spend = round(usd_amt*usd_price,2)


    output = []
    output.append(tuple([inv_date.strftime('%b %d, %Y'),"One-off", str(index + 1),  "stocks", row['ST'], stocks_price, stocks_amt, stocks_spend, inv_period]))
    output.append(tuple([inv_date.strftime('%b %d, %Y'),"One-off", str(index + 1),  "cbonds",row['CB'], cbonds_price, cbonds_amt, cbonds_spend, inv_period]))
    output.append(tuple([inv_date.strftime('%b %d, %Y'),"One-off", str(index + 1),  "sbonds", row['PB'], sbonds_price, sbonds_amt, sbonds_spend, inv_period]))
    output.append(tuple([inv_date.strftime('%b %d, %Y'),"One-off", str(index + 1),  "gold", row['GO'], gold_price, gold_amt, gold_spend, inv_period]))
    output.append(tuple([inv_date.strftime('%b %d, %Y'),"One-off", str(index + 1),  "cash", row['CA'], usd_price, usd_amt, usd_spend, inv_period]))

    with open(methodsFile, 'a+', newline = '') as file:
        writer = csv.writer(file)
        for x in output:
            writer.writerow(x)

print("one-off, no rebalance done")

one-off, no rebalance done


In [132]:
oneoff_df = pd.read_csv('trading_methodologies.csv')
oneoff_df = oneoff_df[oneoff_df['Trading method'] == 'One-off']
oneoff_df.columns

Index(['Date', 'Trading method', 'Allocation no', 'Asset', 'Weight of asset',
       'Asset price', 'Amount to buy', 'Total spending', 'Investment period'],
      dtype='object')

In [133]:
#one-off, rebalance
oneoff_df = pd.read_csv('trading_methodologies2.csv')
oneoff_df = oneoff_df[oneoff_df['Trading method'] == 'One-off']
oneoff_rebal = oneoff_df.copy()
oneoff_rebal =  oneoff_rebal.replace({"One-off": "Oneoff-rebal"})
assets = ['stocks', 'cbonds', 'sbonds', 'gold', 'cash']

def new_quantities(allocation, date, prec_portofolio):
    stocks_w = allocation[0]
    cbonds_w = allocation[1]
    sbonds_w = allocation[2]
    gold_w = allocation[3]
    usd_w = allocation[4]

    amount_to_buy = list(prec_portofolio['Amount to buy'])
    stocks_q = amount_to_buy[0]
    cbonds_q = amount_to_buy[1]
    sbonds_q = amount_to_buy[2]
    gold_q = amount_to_buy[3]
    usd_q = amount_to_buy[4]

    stocks_index = stocks_df.loc[stocks_df['Date'] == date.strftime('%b %d, %Y')]
    stocks_price = stocks_index.iloc[0]['Price']
    cbonds_index = cbonds_df.loc[cbonds_df['Date'] == date.strftime('%b %d, %Y')]
    cbonds_price = cbonds_index.iloc[0]['Price']
    sbonds_index = sbonds_df.loc[sbonds_df['Date'] == date.strftime('%b %d, %Y')]
    sbonds_price = sbonds_index.iloc[0]['Price']
    gold_index = gold_df.loc[gold_df['Date'] == date.strftime('%b %d, %Y')]
    gold_price = gold_index.iloc[0]['Price']
    usd_price = 1

    prices = [stocks_price, cbonds_price, sbonds_price, gold_price, usd_price]

    #How much money do I have in total
    total_usd = stocks_q*stocks_price + cbonds_q*cbonds_price + sbonds_q*sbonds_price + gold_q*gold_price + usd_q

    def new_quantity(w, price, total):
        quantity = total*w // price
        return quantity

    new_quantities = []
    for i in range(len(allocation)):
        new_quantities.append(new_quantity(allocation[i], prices[i], total_usd))

    return new_quantities, prices


def rebalancing(allocation_no, allocation, inv_date, nb_months, portofolio):
    rebal_portofolio1 = rebal_portofolio.append(portofolio)
    first_year, first_month = inv_date.year, inv_date.month
    dates = [datetime(first_year, first_month, 15)]
    #Dates à choisir en fonction de s'il existe ou pas
    for i in range(nb_months-1):
        month = inv_date.month + i + 1
        year = first_year + month // 12
        month = month % 12
        if month == 0:
            month = 12
        dates.append(datetime(year, month, 15))

    for date in dates:
        prec_portofolio = portofolio[-5:]
        new_portofolio = pd.DataFrame(columns=['Date','Trading method', 'Allocation no', 'Asset', 'Weight of asset', 'Asset price', 'Amount to buy', 'Total spending', 'Investment period'])
        new_quantity, prices = new_quantities(allocation, date, prec_portofolio)
        
        new_portofolio['Date'] = [date.strftime('%b %d, %Y') for _ in range(5)]
        new_portofolio['Trading method'] = ["Oneoff-rebal" for _ in range(5)]
        new_portofolio["Allocation no"] = [allocation_no for _ in range(5)]
        new_portofolio['Asset'] = assets
        new_portofolio['Weight of asset'] = allocation
        new_portofolio['Asset price'] = prices
        new_portofolio['Amount to buy'] = new_quantity
        new_portofolio['Total spending'] = np.multiply(prices, new_quantity)
        new_portofolio['Investment period'] = nb_months

        rebal_portofolio1 = rebal_portofolio1.append(new_portofolio)

    return rebal_portofolio1


In [138]:
#making the real portofolio
rebal_portofolio = pd.DataFrame(columns=['Date','Trading method', 'Allocation no', 'Asset', 'Weight of asset', 'Asset price', 'Amount to buy', 'Total spending', 'Investment period'])

for iteration, row in portfolio_df.iterrows():
    print('.', flush=True, end='')
    initial_portofolio = oneoff_df.loc[oneoff_df['Allocation no'] == iteration + 1]
    rebal_portofolio = rebalancing(iteration + 1, list(row[1:]), inv_date, 3, initial_portofolio)

rebal_portofolio = rebal_portofolio.replace({"One-off": "Oneoff-rebal"}) 

print("one-off with rebalance done")

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [141]:
rebal_portofolio.tail(50)
#Sort the df by allocation number

#Write all in the csv file

,Date,Trading method,Allocation no,Asset,Weight of asset,Asset price,Amount to buy,Total spending,Investment period
0,"Aug 15, 2020",Oneoff-rebal,10624,stocks,0.00,186.69,0.0,0.0,3
1,"Aug 15, 2020",Oneoff-rebal,10624,cbonds,0.00,83.04,0.0,0.0,3
2,"Aug 15, 2020",Oneoff-rebal,10624,sbonds,0.00,261.90,0.0,0.0,3
3,"Aug 15, 2020",Oneoff-rebal,10624,gold,0.10,166.62,60.0,9997.2,3
4,"Aug 15, 2020",Oneoff-rebal,10624,cash,0.90,1.00,89997.0,89997.0,3
0,"Sep 15, 2020",Oneoff-rebal,10624,stocks,0.00,188.44,0.0,0.0,3
1,"Sep 15, 2020",Oneoff-rebal,10624,cbonds,0.00,83.04,0.0,0.0,3
2,"Sep 15, 2020",Oneoff-rebal,10624,sbonds,0.00,261.90,0.0,0.0,3
3,"Sep 15, 2020",Oneoff-rebal,10624,gold,0.10,166.62,60.0,9997.2,3
4,"Sep 15, 2020",Oneoff-rebal,10624,cash,0.90,1.00,89997.0,89997.0,3


In [40]:
for iter, rows in list(portfolio_df.iterrows())[:10]:
    print(iter, list(rows))

0 [0.0, 1.0, 0.0, 0.0, 0.0, 0.0]
1 [1.0, 0.95, 0.05, 0.0, 0.0, 0.0]
2 [2.0, 0.95, 0.0, 0.05, 0.0, 0.0]
3 [3.0, 0.95, 0.0, 0.0, 0.05, 0.0]
4 [4.0, 0.95, 0.0, 0.0, 0.0, 0.05]
5 [5.0, 0.9, 0.1, 0.0, 0.0, 0.0]
6 [6.0, 0.9, 0.05, 0.05, 0.0, 0.0]
7 [7.0, 0.9, 0.05, 0.0, 0.05, 0.0]
8 [8.0, 0.9, 0.05, 0.0, 0.0, 0.05]
9 [9.0, 0.9, 0.0, 0.1, 0.0, 0.0]


In [25]:
datetime(2020, 12, 1).month

12

In [18]:
stocks_index = stocks_df.loc[stocks_df['Date'] == datetime(2020, 1, 15).strftime('%b %d, %Y')]
stocks_price = stocks_index.iloc[0]['Price']
print(stocks_price)

196.77


In [33]:
['f' for _ in range(5)]

['f', 'f', 'f', 'f', 'f']

In [ ]:
    



#DCA, no rebalance
monthly_inv = math.floor(amount/inv_period)

        #check data availability for the entire period
end_date = inv_date + relativedelta(months = inv_period)

if stocks_df.loc[stocks_df['Date'] == end_date.strftime('%b %d, %Y')].empty:
        print("no data for the entire period, stocks")
if sbonds_df.loc[sbonds_df['Date'] == end_date.strftime('%b %d, %Y')].empty:
        print("no data for the entire period, sbonds")
if cbonds_df.loc[cbonds_df['Date'] == end_date.strftime('%b %d, %Y')].empty:
        print("no data for the entire period, cbonds")
if gold_df.loc[gold_df['Date'] == end_date.strftime('%b %d, %Y')].empty:
        print("no data for the entire period, gold")

for index, row in portfolio_df.iterrows():

    allocation_index = row[0]
    stocks_rate = row['ST']
    cbonds_rate = row['CB']
    sbonds_rate = row['PB']
    gold_rate = row['GO']
    usd_rate = row['CA']

    stocks_monthly = stocks_rate*monthly_inv
    cbonds_monthly = cbonds_rate*monthly_inv
    sbonds_monthly = sbonds_rate*monthly_inv
    gold_monthly = gold_rate*monthly_inv
    usd_monthly = usd_rate*monthly_inv

    stocks_sum = 0
    cbonds_sum = 0
    sbonds_sum = 0
    gold_sum = 0
    usd_sum = 0

    stocks_cumm = 0
    cbonds_cumm = 0
    sbonds_cumm = 0
    gold_cumm = 0
    usd_cumm = 0
    
    for x in range(0, inv_period):
            
        curr_date = inv_date + relativedelta(months = x)

        stocks_price =  stocks_df.loc[stocks_df['Date'] == curr_date.strftime('%b %d, %Y')].iloc[0]['Price']
        cbonds_price =  cbonds_df.loc[cbonds_df['Date'] == curr_date.strftime('%b %d, %Y')].iloc[0]['Price']
        sbonds_price =  sbonds_df.loc[sbonds_df['Date'] == curr_date.strftime('%b %d, %Y')].iloc[0]['Price']
        gold_price =  gold_df.loc[gold_df['Date'] == curr_date.strftime('%b %d, %Y')].iloc[0]['Price']
        usd_price = 1

        stocks_amt = math.floor(stocks_monthly/stocks_price)
        stocks_sum += stocks_amt
        cbonds_amt = math.floor(cbonds_monthly/cbonds_price)
        cbonds_sum += cbonds_amt
        sbonds_amt = math.floor(sbonds_monthly/sbonds_price)
        sbonds_sum += sbonds_amt
        gold_amt = math.floor(gold_monthly/gold_price)
        gold_sum += gold_amt
        usd_amt = math.floor(usd_monthly/usd_price)
        usd_sum += usd_amt

         #amount of money to invest in each

        stocks_spend = round(stocks_amt*stocks_price,2)
        stocks_cumm += stocks_spend
        cbonds_spend = round(cbonds_amt*cbonds_price,2)
        cbonds_cumm += cbonds_spend
        sbonds_spend = round(sbonds_amt*sbonds_price,2)
        sbonds_cumm += sbonds_spend
        gold_spend = round(gold_amt*gold_price,2)
        gold_cumm += gold_spend
        usd_spend = round(usd_amt*usd_price,2)
        usd_cumm += gold_spend

        output = []
        output.append(tuple([inv_date.strftime('%b %d, %Y'),"DCA", str(index + 1),  "stocks", row['ST'], stocks_price, stocks_sum, stocks_cumm, inv_period]))
        output.append(tuple([inv_date.strftime('%b %d, %Y'),"DCA",  str(index + 1),  "sbonds",row['CB'], cbonds_price, cbonds_sum, cbonds_cumm, inv_period]))
        output.append(tuple([inv_date.strftime('%b %d, %Y'),"DCA", str(index + 1),  "sbonds", row['PB'], sbonds_price, sbonds_sum, sbonds_cumm, inv_period]))
        output.append(tuple([inv_date.strftime('%b %d, %Y'),"DCA", str(index + 1),  "gold", row['GO'], gold_price, gold_sum, gold_cumm, inv_period]))
        output.append(tuple([inv_date.strftime('%b %d, %Y'),"DCA", str(index + 1),  "cash", row['CA'], usd_price, usd_sum, usd_cumm, inv_period]))

        with open(methodsFile, 'a+', newline = '') as file:
                writer = csv.writer(file)
                for x in output:
                    writer.writerow(x)
                    
print("DCA, no rebalance done")       
#DCA, rebalance
